In [1]:
import cv2
import os
import numpy as np

def auto_select_roi(image):
    """
    Automatically select a bounding box enclosing a circular region in the image.
    """
    # Resize the image to a smaller size for faster processing
    scale_percent = 30 # You can adjust this percentage
    width = int(image.shape[1] * scale_percent / 100)
    height = int(image.shape[0] * scale_percent / 100)
    dim = (width, height)
    resized_image = cv2.resize(image, dim, interpolation=cv2.INTER_AREA)
    
    # Convert resized image to grayscale
    gray = cv2.cvtColor(resized_image, cv2.COLOR_BGR2GRAY)
    
    # Apply Gaussian blur to reduce noise
    blurred = cv2.GaussianBlur(gray, (5, 5), 0)
    
    # Define the expected radius range for the circular region (adjusted for 75% of the original size)
    min_radius = int(resized_image.shape[0] * 0.3)  # 30% of the image height
    max_radius = int(resized_image.shape[0] * 0.6)  # 40% of the image height
    
    # Detect circles using Hough Circle Transform
    circles = cv2.HoughCircles(blurred, cv2.HOUGH_GRADIENT, dp=1, minDist=20,
                               param1=50, param2=30, minRadius=min_radius, maxRadius=max_radius)
    
    if circles is not None:
        # Convert circle parameters to integer
        circles = np.round(circles[0, :]).astype("int")
        
        # Get the bounding box enclosing the detected circle
        x, y, r = circles[0]
        
        # Calculate the bounding box dimensions (scaled to 75% of the circle radius)
        x1, y1 = int((x - r*1.1) * 100 / scale_percent), int((y - r*1.1) * 100 / scale_percent)
        x2, y2 = int((x + r*1.1) * 100 / scale_percent), int((y + r*1.1) * 100 / scale_percent)
        
        return [x1, y1, x2 - x1, y2 - y1]  # Return bounding box coordinates [x, y, w, h]
    else:
        return None

def crop_image(image, x, y, w, h):
    """
    Crop the input image using the provided coordinates.
    """
    return image[y:y+h, x:x+w]

def crop_it(input_folder, output_folder):
    """
    Main function to read tif files in a folder, automatically select a circular ROI,
    crop each image, and save them in a new folder.
    """
    # Create the output folder if it doesn't exist
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    # Get the sorted list of files in the input folder
    sorted_files = sorted(os.listdir(input_folder))
    
    for filename in sorted_files:
        if filename.endswith(".tif"):
            file_path = os.path.join(input_folder, filename)
            image = cv2.imread(file_path)
            if image is not None:
                roi = auto_select_roi(image)
                if roi is not None:
                    cropped_image = crop_image(image, roi[0], roi[1], roi[2], roi[3])
                    
                    # Save cropped image in the output folder
                    output_file_path = os.path.join(output_folder, filename)
                    cv2.imwrite(output_file_path, cropped_image)
                    print(f"Saved {output_file_path}")

In [2]:
input_folder = "/media/saab/f7ee81f1-4052-4c44-b470-0a4a650ee479/SGH_IMCB_NTU_Main/Raw_Data/Validation Dataset/STR192-14 Lung TMA Post-HE CD3 IHC"
output_folder = "/media/saab/f7ee81f1-4052-4c44-b470-0a4a650ee479/SGH_IMCB_NTU_Main/Raw_Data/Validation Dataset/IHC_TIFF_Same_section_Cropped"
crop_it(input_folder, output_folder)

Saved /media/saab/f7ee81f1-4052-4c44-b470-0a4a650ee479/SGH_IMCB_NTU_Main/Raw_Data/Validation Dataset/IHC_TIFF_Same_section_Cropped/STR192-14 Lung TMA Post-HE CD3 IHC_s01.tif
Saved /media/saab/f7ee81f1-4052-4c44-b470-0a4a650ee479/SGH_IMCB_NTU_Main/Raw_Data/Validation Dataset/IHC_TIFF_Same_section_Cropped/STR192-14 Lung TMA Post-HE CD3 IHC_s02.tif
Saved /media/saab/f7ee81f1-4052-4c44-b470-0a4a650ee479/SGH_IMCB_NTU_Main/Raw_Data/Validation Dataset/IHC_TIFF_Same_section_Cropped/STR192-14 Lung TMA Post-HE CD3 IHC_s03.tif
Saved /media/saab/f7ee81f1-4052-4c44-b470-0a4a650ee479/SGH_IMCB_NTU_Main/Raw_Data/Validation Dataset/IHC_TIFF_Same_section_Cropped/STR192-14 Lung TMA Post-HE CD3 IHC_s04.tif
Saved /media/saab/f7ee81f1-4052-4c44-b470-0a4a650ee479/SGH_IMCB_NTU_Main/Raw_Data/Validation Dataset/IHC_TIFF_Same_section_Cropped/STR192-14 Lung TMA Post-HE CD3 IHC_s05.tif
Saved /media/saab/f7ee81f1-4052-4c44-b470-0a4a650ee479/SGH_IMCB_NTU_Main/Raw_Data/Validation Dataset/IHC_TIFF_Same_section_Cropped

error: OpenCV(4.9.0) /io/opencv/modules/imgcodecs/src/loadsave.cpp:786: error: (-215:Assertion failed) !_img.empty() in function 'imwrite'
